In [2]:
from google.colab import files
uploaded = files.upload()


Saving fraud_dataset.csv to fraud_dataset.csv


In [3]:
import pandas as pd
df = pd.read_csv('fraud_dataset.csv')
df.head()
print(df.head())

   transaction_id  transaction_amount       location  merchant  age gender  \
0               1              1000.0       New York  ABC Corp   35      M   
1               2               500.0        Chicago   XYZ Inc   45      F   
2               3              2000.0    Los Angeles  ABC Corp   28      M   
3               4              1500.0  San Francisco   XYZ Inc   30      F   
4               5               800.0        Chicago  ABC Corp   50      F   

   fraud_label  
0            0  
1            0  
2            1  
3            0  
4            0  


In [6]:
from sklearn.preprocessing import LabelEncoder

# Select relevant columns
features = ['transaction_amount', 'location', 'age', 'gender']
target = 'fraud_label'

df = df[features + [target]].dropna()

# Encode categorical features
label_encoders = {}
for col in ['location', 'gender']:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

X = df[features]
y = df[target]



In [7]:
from sklearn.model_selection import train_test_split

X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.375, stratify=y_temp, random_state=42)  # 0.375 * 0.8 ≈ 0.3


In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, roc_auc_score

models = {
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'Random Forest': RandomForestClassifier(n_estimators=100),
    'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='logloss')
}

best_model = None
best_auc = 0

for name, model in models.items():
    model.fit(X_train, y_train)
    preds = model.predict_proba(X_val)[:, 1]
    auc = roc_auc_score(y_val, preds)
    print(f"{name} AUC: {auc:.4f}")
    if auc > best_auc:
        best_auc = auc
        best_model = model


Logistic Regression AUC: 0.9429
Random Forest AUC: 0.9524
XGBoost AUC: 0.9048


/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [22:04:12] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [10]:
import joblib

joblib.dump(best_model, 'best_model.pkl')
joblib.dump(label_encoders, 'label_encoders.pkl')


['label_encoders.pkl']